In [72]:
import itertools
import matplotlib.pyplot as plt
import json
import seaborn
from pathlib import Path
import numpy as np
from statistics import mean, median
import pandas as pd

In [73]:
def split_simulations_with_and_without_expansion(simulations):
    with_expansions = []
    without_expansions = []
    for simulations_per_model in simulations:
        for simulations_per_game in simulations_per_model:
            for simulations_per_move in simulations_per_game:
                with_expansions_for_move, without_expansions_for_move = simulations_per_move
                with_expansions.append(with_expansions_for_move)
                without_expansions.append(without_expansions_for_move)
    return with_expansions, without_expansions
            
game = "othello"
with_state_evaluator = False
results = []
complex_simulations = []
simple_simulations = []
for result_file in Path(f"{game}/complex_rollouts/{'with' if with_state_evaluator else 'without'}_state_evaluator/").iterdir():
    with open(result_file) as f:
        result = json.load(f)
        results.append(result["results"])
        complex_simulations.append(result["complex_simulations"])
        simple_simulations.append(result["simple_simulations"])
results = np.array(results)
wins = results[:, 0:2, :].reshape((20, 4))# + 0.5 * results[:, 1, :]
wins = pd.DataFrame(wins, columns=["first_wins", "second_wins", "first_draws", "second_draws"])
wins = wins[["first_wins", "first_draws", "second_wins", "second_draws"]]
wins.insert(0, "model", np.arange(20))
wins.insert(5, "overall_wins", (wins["first_wins"] + wins["second_wins"]) / 2)
wins.insert(6, "overall_draws", (wins["first_draws"] + wins["second_draws"]) / 2)
wins = wins.set_index("model")

In [74]:
index = pd.MultiIndex.from_product([["first", "second", "overall"], ["win", "draw"], ["value", "sem"]])
wins.insert(1, "first_wins_sem", -1)
wins.insert(3, "first_draws_sem", -1)
wins.insert(5, "second_wins_sem", -1)
wins.insert(7, "second_draws_sem", -1)
wins.insert(9, "overall_wins_sem", -1)
wins.insert(11, "overall_draws_sem", -1)
wins.columns = index
for player, type in itertools.product(["first", "second", "overall"], ["win", "draw"]):
    wins[player, type, "sem"] = np.sqrt(wins[player, type, "value"] * (1 - wins[player, type, "value"]) / (240 if player == "overall" else 120))
totals = []
for player, type in itertools.product(["first", "second", "overall"], ["win", "draw"]):
    total = np.mean(wins[player, type, "value"])
    totals.append(total)
    totals.append(np.sqrt(total * (1 - total) / (20 * (240 if player == "overall" else 120))))
wins.loc["Total", :] = totals
wins.index = wins.index.rename("Model")

In [75]:
latex_wins = pd.DataFrame(columns=pd.MultiIndex.from_product([["first", "second", "overall"], ["win", "draw"]]))
latex_wins.index = latex_wins.index.rename("Model")
def f(df):
    def inner(player, type):
        return f'${df[player, type, "value"]:.2f} \pm {df[player, type, "sem"] * 1.96:.2f}$'
    return [inner(player, type) for player in ["first", "second", "overall"] for type in ["win", "draw"]]
latex_wins[[("first", "win"), ("first", "draw"), ("second", "win"), ("second", "draw"), ("overall", "win"), ("overall", "draw")]] = wins.apply(func=f, axis="columns", result_type="expand")
# latex_wins = latex_wins.rename(columns={"first": "First player win ratio", "second": "Second player win ratio", "overall": "Overall win ratio", "value": "Value", "sem": "SEM"})
with open(f"{game}_complex_rollouts_{'with' if with_state_evaluator else 'without'}_state_evaluator.txt", "w", encoding="utf-8") as f:
   latex_wins.to_latex(f, escape=False)
with open(f"{game}_complex_rollouts_overall_total_{'with' if with_state_evaluator else 'without'}_state_evaluator.txt", "w", encoding="utf-8") as f:
    latex_wins.loc["Total", "overall"].to_latex(f, escape=False, header=False)
latex_wins

first                            second                   \
                   win             draw              win             draw   
Model                                                                       
0      $0.32 \pm 0.08$  $0.05 \pm 0.04$  $0.45 \pm 0.09$  $0.07 \pm 0.04$   
1      $0.45 \pm 0.09$  $0.05 \pm 0.04$  $0.43 \pm 0.09$  $0.05 \pm 0.04$   
2      $0.33 \pm 0.08$  $0.08 \pm 0.05$  $0.49 \pm 0.09$  $0.04 \pm 0.04$   
3      $0.38 \pm 0.09$  $0.05 \pm 0.04$  $0.38 \pm 0.09$  $0.03 \pm 0.03$   
4      $0.42 \pm 0.09$  $0.04 \pm 0.04$  $0.40 \pm 0.09$  $0.08 \pm 0.05$   
5      $0.43 \pm 0.09$  $0.03 \pm 0.03$  $0.50 \pm 0.09$  $0.09 \pm 0.05$   
6      $0.40 \pm 0.09$  $0.04 \pm 0.04$  $0.42 \pm 0.09$  $0.03 \pm 0.03$   
7      $0.48 \pm 0.09$  $0.04 \pm 0.04$  $0.46 \pm 0.09$  $0.07 \pm 0.05$   
8      $0.37 \pm 0.09$  $0.04 \pm 0.04$  $0.43 \pm 0.09$  $0.07 \pm 0.05$   
9      $0.47 \pm 0.09$  $0.06 \pm 0.04$  $0.45 \pm 0.09$  $0.04 \pm 0.04$   
10     $0.40 \pm 0.09$  $0.04 \pm 0.04$  $0.43 \pm 0.09$  $0.03 \pm 0.03$   
11     $0.45 \pm 0.09$  $0.03 \pm 0.03$  $0.40 \pm 0.09$  $0.07 \pm 0.04$   
12     $0.43 \pm 0.09$  $0.03 \pm 0.03$  $0.38 \pm 0.09$  $0.03 \pm 0.03$   
13     $0.48 \pm 0.09$  $0.05 \pm 0.04$  $0.41 \pm 0.09$  $0.07 \pm 0.04$   
14     $0.42 \pm 0.09$  $0.07 \pm 0.04$  $0.41 \pm 0.09$  $0.02 \pm 0.02$   
15     $0.35 \pm 0.09$  $0.09 \pm 0.05$  $0.41 \pm 0.09$  $0.03 \pm 0.03$   
16     $0.44 \pm 0.09$  $0.05 \pm 0.04$  $0.46 \pm 0.09$  $0.03 \pm 0.03$   
17     $0.43 \pm 0.09$  $0.06 \pm 0.04$  $0.40 \pm 0.09$  $0.07 \pm 0.04$   
18     $0.48 \pm 0.09$  $0.04 \pm 0.04$  $0.40 \pm 0.09$  $0.06 \pm 0.04$   
19     $0.45 \pm 0.09$  $0.04 \pm 0.04$  $0.42 \pm 0.09$  $0.06 \pm 0.04$   
Total  $0.42 \pm 0.02$  $0.05 \pm 0.01$  $0.43 \pm 0.02$  $0.05 \pm 0.01$   

               overall                   
                   win             draw  
Model                                    
0      $0.38 \pm 0.06$  $0.06 \pm 0.03$  
1      $0.44 \pm 0.06$  $0.05 \pm 0.03$  
2      $0.41 \pm 0.06$  $0.06 \pm 0.03$  
3      $0.38 \pm 0.06$  $0.04 \pm 0.02$  
4      $0.41 \pm 0.06$  $0.06 \pm 0.03$  
5      $0.47 \pm 0.06$  $0.06 \pm 0.03$  
6      $0.41 \pm 0.06$  $0.03 \pm 0.02$  
7      $0.47 \pm 0.06$  $0.06 \pm 0.03$  
8      $0.40 \pm 0.06$  $0.06 \pm 0.03$  
9      $0.46 \pm 0.06$  $0.05 \pm 0.03$  
10     $0.42 \pm 0.06$  $0.04 \pm 0.02$  
11     $0.43 \pm 0.06$  $0.05 \pm 0.03$  
12     $0.41 \pm 0.06$  $0.03 \pm 0.02$  
13     $0.44 \pm 0.06$  $0.06 \pm 0.03$  
14     $0.41 \pm 0.06$  $0.04 \pm 0.03$  
15     $0.38 \pm 0.06$  $0.06 \pm 0.03$  
16     $0.45 \pm 0.06$  $0.04 \pm 0.03$  
17     $0.42 \pm 0.06$  $0.06 \pm 0.03$  
18     $0.44 \pm 0.06$  $0.05 \pm 0.03$  
19     $0.44 \pm 0.06$  $0.05 \pm 0.03$  
Total  $0.42 \pm 0.01$  $0.05 \pm 0.01$

In [76]:
complex_simulations_with_expansion, complex_simulations_without_expansion = split_simulations_with_and_without_expansion(complex_simulations)
simple_simulations_with_expansion, simple_simulations_without_expansion = split_simulations_with_and_without_expansion(simple_simulations)
average_complex_simulations = mean(complex_simulations_with_expansion), mean(complex_simulations_without_expansion)
average_simple_simulations = mean(simple_simulations_with_expansion), mean(simple_simulations_without_expansion)
print(average_complex_simulations)
print(average_simple_simulations)
simulations = pd.DataFrame.from_dict({"Random rollouts": [mean(simple_simulations_with_expansion), sum(average_simple_simulations)], "Policy network rollouts": [mean(complex_simulations_with_expansion), sum(average_complex_simulations)]}, orient="index", columns=["Expansions", "Simulations"])
with open(f"{game}_complex_rollouts_simulations_{'with' if with_state_evaluator else 'without'}_state_evaluator.txt", "w", encoding="utf-8") as f:
    simulations.to_latex(f, float_format="{:.0f}".format)

(59.00146599528332, 16578.349544266683)
(344.4308202154101, 19529.9628959276)
